In [1]:
pip install scikit-fuzzy

   ---------------------------------------- 0.0/920.8 kB ? eta -:--:--
   --------------------------------------- 920.8/920.8 kB 14.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: C:\Users\aleix\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

In [9]:
import pandas as pd
import numpy as np

# Llista dels 51 estats dels EUA (noms complets)
estats_usa = [
    "Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut", "Delaware",
    "District of Columbia", "Florida", "Georgia", "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa",
    "Kansas", "Kentucky", "Louisiana", "Maine", "Maryland", "Massachusetts", "Michigan", "Minnesota",
    "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada", "New Hampshire", "New Jersey", "New Mexico",
    "New York", "North Carolina", "North Dakota", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Rhode Island",
    "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Vermont", "Virginia", "Washington",
    "West Virginia", "Wisconsin", "Wyoming"
]
# Genera dades aleatòries realistes (ajustables)
np.random.seed(42)  # Per reprodibilitat

# Genera valors aleatoris per hospitalitzats, morts i contagiats
df_epidemiologic = pd.DataFrame({
    "state": estats_usa,
    "hospitalitzats": np.random.randint(50, 2000, size=51),  # Exemple: entre 50 i 2000
    "morts": np.random.randint(5, 500, size=51),            # Exemple: entre 5 i 500
    "contagiats": np.random.randint(1000, 50000, size=51)   # Exemple: entre 1000 i 50000
})

# Mostra les primeres files
print(df_epidemiologic.head())

        state  hospitalitzats  morts  contagiats
0     Alabama            1176     55       34763
1      Alaska            1509    368       33606
2     Arizona             910     59       12534
3    Arkansas            1344    248       30127
4  California            1180    324       41397


In [10]:
import pandas as pd

# Llegeix el fitxer demogràfic (ajusta la ruta i el delimitador)
df_demografic = pd.read_csv("C:/Users/aleix/Desktop/UNI/6eQuadrimestre/PAID/IDSS/IDSS-Pandemic_Control/data/preprocessed/dataMatrix/static_stateMatrix.csv")

# Selecciona les columnes demogràfiques necessàries
columnes_demografia = [
    "state", 
    "population_state",
    "pop_0-9", "pop_10-19", "pop_20-29", "pop_30-39",
    "pop_40-49", "pop_50-59", "pop_60-69", "pop_70-79", "pop_80+"
]
df_demografic = df_demografic[columnes_demografia]

In [11]:
# Fusiona els dos DataFrames per la columna "state"
df_combinat = pd.merge(
    df_demografic, 
    df_epidemiologic, 
    on="state", 
    how="inner"  # Només estats presents a tots dos fitxers
)

# Mostra les primeres files (opcional)
print(df_combinat.head())

        state  population_state   pop_0-9  pop_10-19  pop_20-29  pop_30-39  \
0     Alabama           4887871  0.120933   0.128108   0.136099   0.124291   
1      Alaska            737438  0.142617   0.130210   0.155501   0.147626   
2     Arizona           7171646  0.123774   0.132496   0.140854   0.129224   
3    Arkansas           3013825  0.127289   0.132222   0.135421   0.126671   
4  California          39557045  0.124635   0.128760   0.147575   0.143412   

   pop_40-49  pop_50-59  pop_60-69  pop_70-79   pop_80+  hospitalitzats  \
0   0.123013   0.133249   0.120537   0.075243  0.038527            1176   
1   0.117347   0.127204   0.109072   0.050675  0.019748            1509   
2   0.119039   0.120016   0.113492   0.080306  0.040799             910   
3   0.119787   0.126968   0.115755   0.075878  0.040010            1344   
4   0.128138   0.126974   0.104040   0.060900  0.035566            1180   

   morts  contagiats  
0     55       34763  
1    368       33606  
2     59   

In [18]:
defuncions = ctrl.Antecedent(np.arange(0, 501, 1), 'defuncions')  # Exemple: 0-500 morts
infectats = ctrl.Antecedent(np.arange(0, 100001, 1), 'infectats')  # Exemple: 0-100k infectats
hospitalitzats = ctrl.Antecedent(np.arange(0, 10001, 1), 'hospitalitzats')  # Exemple: 0-10k hospitalitzats


# Variable de sortida
risc = ctrl.Consequent(np.arange(0, 11, 1), 'risc')

# Funcions d'afiliació (exemple)
defuncions.automf(3, names=['Baix', 'Mitjà', 'Alt'])
infectats.automf(3, names=['Baix', 'Mitjà', 'Alt'])
hospitalitzats.automf(3, names=['Baix', 'Mitjà', 'Alt'])

risc['Baix'] = fuzz.trimf(risc.universe, [0, 0, 5])
risc['Mitjà'] = fuzz.trimf(risc.universe, [0, 5, 10])
risc['Alt'] = fuzz.trimf(risc.universe, [5, 10, 10])

# Regles difuses (exemple)
regla1 = ctrl.Rule(defuncions['Alt'] | infectats['Alt'] | hospitalitzats['Alt'], risc['Alt'])
regla2 = ctrl.Rule(defuncions['Mitjà'] & infectats['Mitjà'], risc['Mitjà'])
regla3 = ctrl.Rule(defuncions['Baix'] & infectats['Baix'] & hospitalitzats['Baix'], risc['Baix'])

sistema_risc = ctrl.ControlSystem([regla1, regla2, regla3])
calculador_risc = ctrl.ControlSystemSimulation(sistema_risc)

In [21]:
resultats = []
for _, estat in df_combinat.iterrows():
    try:
        calculador_risc.input['defuncions'] = estat['morts']
        calculador_risc.input['infectats'] = estat['contagiats']
        calculador_risc.input['hospitalitzats'] = estat['hospitalitzats']
        
        calculador_risc.compute()
        resultats.append(calculador_risc.output['risc'])
    except Exception as e:
        print(f"Error a l'estat {estat['state']}: {e}")
        resultats.append(np.nan)  # Afegeix NaN si hi ha error

# Afegeix els resultats al DataFrame
df_combinat['risc'] = resultats
print(df_combinat[['state', 'risc']])

                   state      risc
0                Alabama  4.352421
1                 Alaska  5.533006
2                Arizona  3.479155
3               Arkansas  4.995570
4             California  5.223613
5               Colorado  4.491967
6            Connecticut  7.887089
7               Delaware  5.189115
8   District of Columbia  4.504773
9                Florida  3.048313
10               Georgia  5.319452
11                Hawaii  4.172460
12                 Idaho  5.085925
13              Illinois  6.403960
14               Indiana  4.143234
15                  Iowa  5.389974
16                Kansas  2.738277
17              Kentucky  4.992242
18             Louisiana  5.096328
19                 Maine  5.376812
20              Maryland  4.739319
21         Massachusetts  5.760706
22              Michigan  5.339552
23             Minnesota  2.647217
24           Mississippi  7.259238
25              Missouri  7.508470
26               Montana  6.931332
27              Nebr